# Kỹ thuật đo lường và cải tiến hiệu suất - Performance Measurement and Improvement Techniques

Trong xử lý hình ảnh, vì bạn phải xử lý một số lượng lớn các phép toán mỗi giây, nên bắt buộc mã của bạn không chỉ cung cấp giải pháp chính xác mà còn phải cung cấp giải pháp theo cách nhanh nhất. 

Vì vậy, trong chương này, bạn sẽ tìm hiểu:
- Cách đo hiệu suất của code.
- Một số mẹo để cải thiện hiệu suất của mã.
Bạn sẽ thấy các hàm này: `getTickCount`,`getTickFrequency`, v.v.

Ngoài OpenCV, Python còn cung cấp một mô-đun time hữu ích trong việc đo thời gian thực thi. Một cấu hình mô-đun khác giúp có được báo cáo chi tiết về mã, chẳng hạn như thời gian thực hiện từng hàm trong mã, số lần gọi hàm, v.v. Nhưng nếu bạn đang sử dụng IPython, tất cả các tính năng này đều được tích hợp theo cách thân thiện với người dùng. Chúng ta sẽ xem một số tính năng quan trọng và để biết thêm chi tiết, hãy kiểm tra các liên kết trong phần **OpenCV Tour**

In [1]:
import cv2
import numpy as np

## Đo hiệu suất với OpenCV
Hàm `getTickCount` trả về số chu kỳ xung nhịp sau một sự kiện tham chiếu (như thời điểm máy được BẬT) cho đến thời điểm hàm này được gọi. Vì vậy, nếu bạn gọi nó trước và sau khi thực thi hàm, bạn sẽ nhận được số chu kỳ xung nhịp được sử dụng để thực thi một hàm.
### Cú pháp hàm
```python
ticks = cv2.getTickCount()
```
Giải thích:
- `ticks`: Giá trị trả về là số chu kỳ xung nhịp (tick) kể từ một thời điểm tham chiếu (thường là khi máy tính khởi động).
- Thường dùng để đo thời gian thực thi một đoạn mã bằng cách lấy hiệu giữa hai lần gọi getTickCount().

Hàm `getTickFrequency` trả về tần suất của chu kỳ xung nhịp hoặc số chu kỳ xung nhịp mỗi giây. Vì vậy, để tìm thời gian thực thi tính bằng giây, bạn có thể thực hiện như sau:
### Cú pháp hàm
```python
freq = cv2.getTickFrequency()
```
Giải thích:  
- `freq`: Giá trị trả về là tần số của bộ đếm tick, tức là số chu kỳ tick trên mỗi giây (đơn vị: Hz).
- Thường dùng kết hợp với getTickCount() để tính thời gian thực thi (thời gian = số tick / freq).

Chúng tôi sẽ chứng minh bằng ví dụ sau. Ví dụ sau áp dụng lọc trung vị với các hạt nhân có kích thước lẻ từ 5 đến 49. (Đừng lo lắng về kết quả sẽ như thế nào - đó không phải là mục tiêu của chúng tôi):

> Lưu ý
Bạn có thể làm điều tương tự với mô-đun time. Thay vì `getTickCount`, hãy sử dụng hàm `time.time()`. Sau đó, lấy hiệu số của hai thời gian.

In [7]:
e1 = cv2.getTickCount()
# code
src = cv2.imread("images/violet.jpg", cv2.IMREAD_COLOR)
for i in range(5,49,2):
    src = cv2.medianBlur(src,i)
# end of code
e2 = cv2.getTickCount()
time = (e2 - e1)/ cv2.getTickFrequency()
print("Time taken: {:.6f} seconds".format(time))

Time taken: 0.886391 seconds


## Tối ưu hóa mặc định trong OpenCV

Nhiều hàm OpenCV được tối ưu hóa bằng SSE2, AVX, v.v. Nó cũng chứa mã chưa được tối ưu hóa. Vì vậy, nếu hệ thống của chúng ta hỗ trợ các tính năng này, chúng ta nên khai thác chúng (hầu hết các bộ xử lý hiện đại đều hỗ trợ chúng). Nó được bật theo mặc định trong khi biên dịch. Vì vậy, OpenCV chạy mã đã được tối ưu hóa nếu nó được bật, nếu không, nó sẽ chạy mã chưa được tối ưu hóa. Bạn có thể sử dụng `useOptimized()` để kiểm tra xem nó đã được bật/tắt chưa và `setUseOptimized()` để bật/tắt nó. Hãy cùng xem một ví dụ đơn giản.

In [11]:
cv2.setUseOptimized(False)
cv2.useOptimized()

e1 = cv2.getTickCount()
# code
src = cv2.imread("images/violet.jpg", cv2.IMREAD_COLOR)
for i in range(5,49,2):
    src = cv2.medianBlur(src,i)
# end of code
e2 = cv2.getTickCount()
time = (e2 - e1)/ cv2.getTickFrequency()
print("Time taken: {:.6f} seconds".format(time))

Time taken: 0.860005 seconds


Như bạn thấy, bộ lọc trung vị được tối ưu hóa nhanh hơn so với phiên bản chưa được tối ưu hóa. Nếu bạn kiểm tra nguồn của nó, bạn có thể thấy rằng bộ lọc trung vị được tối ưu hóa SIMD. Vì vậy, bạn có thể sử dụng điều này để bật tối ưu hóa ở đầu mã của mình (hãy nhớ rằng nó được bật theo mặc định).

## Đo lường hiệu suất trong IPython

Đôi khi bạn có thể cần so sánh hiệu suất của hai phép toán tương tự. IPython cung cấp cho bạn lệnh timeit để thực hiện phép toán này. Lệnh này chạy mã nhiều lần để có kết quả chính xác hơn. Một lần nữa, lệnh này phù hợp để đo các dòng mã đơn lẻ.

Ví dụ, bạn có biết phép toán cộng nào sau đây tốt hơn không? 
```python
x = 5; y = x**2  
x = 5; y = x*x
x = np.uint8([5]); y = x*x
x = np.unit8([5]); y = np.square(x)
```
Chúng ta sẽ tìm hiểu bằng lệnh timeit trong shell IPython.

```shell
In [10]: x = 5
 
In [11]: %timeit y=x**2
10000000 loops, best of 3: 73 ns per loop
 
In [12]: %timeit y=x*x
10000000 loops, best of 3: 58.3 ns per loop
 
In [15]: z = np.uint8([5])
 
In [17]: %timeit y=z*z
1000000 loops, best of 3: 1.25 us per loop
 
In [19]: %timeit y=np.square(z)
1000000 loops, best of 3: 1.16 us per loop
```

Bạn có thể thấy rằng, x = 5; y = x*x là nhanh nhất và nhanh hơn khoảng 20 lần so với Numpy. Nếu bạn cũng xem xét việc tạo mảng, nó có thể nhanh hơn tới 100 lần. Tuyệt, phải không? (Các nhà phát triển Numpy đang giải quyết vấn đề này)

> Lưu ý
Các phép toán vô hướng Python nhanh hơn các phép toán vô hướng Numpy. Vì vậy, đối với các phép toán bao gồm một hoặc hai phần tử, vô hướng Python tốt hơn các mảng Numpy. Numpy có lợi thế khi kích thước của mảng lớn hơn một chút.

Chúng ta sẽ thử thêm một ví dụ nữa. Lần này, chúng ta sẽ so sánh hiệu suất của `cv2.countNonZero()` và np.`count_nonzero()` cho cùng một hình ảnh.

```shell
In [35]: %timeit z = cv.countNonZero(img)
100000 loops, best of 3: 15.8 us per loop
 
In [36]: %timeit z = np.count_nonzero(img)
1000 loops, best of 3: 370 us per loop
```

Bạn thấy đấy, hàm OpenCV nhanh hơn hàm Numpy gần 25 lần.

> Lưu ý
Thông thường, hàm OpenCV nhanh hơn hàm Numpy. Vì vậy, đối với cùng một hoạt động, hàm OpenCV được ưu tiên hơn. Tuy nhiên, có thể có ngoại lệ, đặc biệt là khi Numpy hoạt động với chế độ xem thay vì bản sao.

## Thêm lệnh đặc biệt IPython

Có một số lệnh ma thuật khác để đo hiệu suất, profiling, line profiling, đo bộ nhớ, v.v. Tất cả đều được ghi chép đầy đủ. Vì vậy, chỉ có các liên kết đến các tài liệu đó được cung cấp ở đây. Độc giả quan tâm được khuyến nghị dùng thử.

## Kỹ thuật tối ưu hóa hiệu suất

Có một số kỹ thuật và phương pháp mã hóa để khai thác hiệu suất tối đa của Python và Numpy. Chỉ những kỹ thuật và phương pháp có liên quan được ghi chú ở đây và các liên kết được cung cấp đến các nguồn quan trọng. Điều chính cần lưu ý ở đây là, trước tiên hãy thử triển khai thuật toán theo cách đơn giản. Khi nó hoạt động, hãy lập hồ sơ, tìm các nút thắt và tối ưu hóa chúng.

1. Tránh sử dụng vòng lặp trong Python càng nhiều càng tốt, đặc biệt là vòng lặp đôi/ba, v.v. Chúng vốn chậm.
2. Vector hóa thuật toán/mã ở mức tối đa có thể, vì Numpy và OpenCV được tối ưu hóa cho các hoạt động vector.
3. Khai thác tính nhất quán của bộ đệm.
4. Không bao giờ tạo bản sao của một mảng trừ khi cần thiết. Thay vào đó, hãy thử sử dụng chế độ xem. Sao chép mảng là một hoạt động tốn kém.

Nếu mã của bạn vẫn chậm sau khi thực hiện tất cả các thao tác này hoặc nếu việc sử dụng các vòng lặp lớn là không thể tránh khỏi, hãy sử dụng các thư viện bổ sung như Cython để làm cho mã nhanh hơn.